## Read in data

In [1]:
import pandas as pd
x_train = pd.read_csv('X_train.csv')
x_test = pd.read_csv('X_val.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_val.csv')

Isolate variables that each of the feature selection techniques shortlisted.

In [2]:
models_best = ['country', 'team', 'season', 'md', 'outcome', 'win%']
models_forward = ['season', 'md', 'team', 'country', 'avg_starter_age', 'outcome', 'win%', 'gd', 'goals', 'opp_gd', 'Cluster']
models_backward = ['country', 'team', 'season', 'md', 'outcome', 'win%', 'goals', 'gd', 'avg_starter_age']

##Models Best

In [3]:
x_train = x_train[models_best]
x_train.head(3)

,country,team,season,md,outcome,win%
0,4,154,2018,13,1,0.076923
1,4,179,2010,19,1,0.105263
2,2,31,2006,6,0,0.500000


In [4]:
x_test = x_test[models_best]

In [5]:
pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 44.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 29.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.

In [6]:
from pandas.core.window.rolling import numba_notes
#Regression Models
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor as XGBRegressor
#Linear Regression ( https://www.w3schools.com/python/python_ml_multiple_regression.asp )
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
#Ridge Regression ( https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html )
from sklearn import linear_model
ridge = linear_model.Ridge(alpha=.5)
#Lasso Regression ( https://towardsdatascience.com/hyperparameter-tuning-in-lasso-and-ridge-regressions-70a4b158ae6d )
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
lasso = linear_model.Lasso(alpha=0.1)
#KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
#Random Forrest ( https://towardsdatascience.com/random-forest-in-python-24d0893d51c0 )
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
#Decision Trees ( https://medium.com/pursuitnotes/decision-tree-regression-in-6-steps-with-python-1a1c5aa2ee16 )
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
#SVM - SVR
from sklearn import svm
SVM = svm.SVR()
#Gradient Boosting Regressor ( https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor )
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=0)
# create an xgboost regression model: higher performance than gradient boosting
import xgboost as xg
xgb = xg.XGBRegressor()
from interpret import show
from interpret.data import Marginal
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree
ebm = ExplainableBoostingRegressor()


In [7]:
model_names = ['Multiple Regression', 'Ridge Regression', 'Lasso Regression', 'KNeighborsRegressor', 'Random Forrest', 'Decision Trees', 'SVM - SVR', 'Gradient Boosting Regressor', 'XGBoost', 'EBM']
models = [lm, ridge, lasso, neigh, rf, dt, SVM, gbr, xgb, ebm]

Set up loop to train each of the models on the features identified in best subset selection

In [8]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import r2_score
model_eval = []
for model in models:
  model.fit(x_train, y_train.values.ravel())
  predictions = model.predict(x_test)
  type(predictions)
  # Calculate the absolute errors
  errors = abs(predictions - y_test.values.ravel())
  # Calculate model evaluation metrics
  mae = mean_absolute_error(y_test, predictions)
  mse = mean_squared_error(y_test, predictions)
  rmse = sqrt(mse)
  r2 = r2_score(y_test, predictions)
  model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

df = pd.DataFrame(model_eval)

In [9]:
df

,Model,Mean Absolute Error,Mean Squared Error,RMSE,R-squared
0,LinearRegression(),0.054495,0.006179,0.078606,0.395267
1,Ridge(alpha=0.5),0.054494,0.006179,0.078606,0.395271
2,Lasso(alpha=0.1),0.066582,0.009231,0.096081,0.096518
3,KNeighborsRegressor(n_neighbors=2),0.020421,0.001573,0.039664,0.846032
4,"(DecisionTreeRegressor(max_features=1.0, rando...",0.014220,0.000855,0.029244,0.916298
5,DecisionTreeRegressor(),0.016715,0.001137,0.033712,0.888771
6,SVR(),0.079653,0.010223,0.101108,-0.000500
7,([DecisionTreeRegressor(criterion='friedman_ms...,0.032269,0.002538,0.050377,0.751627
8,"XGBRegressor(base_score=None, booster=None, ca...",0.016538,0.000727,0.026963,0.928848
9,ExplainableBoostingRegressor(),0.024351,0.001354,0.036800,0.867461


## Forward

In [12]:
import pandas as pd
x_train = pd.read_csv('X_train.csv')
x_test = pd.read_csv('X_val.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_val.csv')

In [13]:
x_train = x_train[models_forward]
x_train.head(3)

,season,md,team,country,avg_starter_age,outcome,win%,gd,goals,opp_gd,Cluster
0,2018,13,154,4,0.531856,1,0.076923,0.500000,0.2,0.500000,2
1,2010,19,179,4,0.443609,1,0.105263,0.500000,0.1,0.500000,2
2,2006,6,31,2,0.511278,0,0.500000,0.444444,0.0,0.555556,0


In [14]:
x_test = x_test[models_forward]
x_test.head(3)

,season,md,team,country,avg_starter_age,outcome,win%,gd,goals,opp_gd,Cluster
0,2015,24,91,3,0.562753,0,0.208333,0.388889,0.0,0.611111,2
1,2004,8,176,0,0.517544,2,0.375000,0.555556,0.1,0.444444,2
2,2015,3,103,2,0.444079,0,0.333333,0.388889,0.1,0.611111,2


Set up loop to train each of the models on the features identified in forward selection

In [15]:
model_eval = []
for model in models:
  model.fit(x_train, y_train.values.ravel())
  predictions = model.predict(x_test)
  type(predictions)
  # Calculate the absolute errors
  errors = abs(predictions - y_test.values.ravel())
  # Calculate model evaluation metrics
  mae = mean_absolute_error(y_test, predictions)
  mse = mean_squared_error(y_test, predictions)
  rmse = sqrt(mse)
  r2 = r2_score(y_test, predictions)
  model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

df2 = pd.DataFrame(model_eval)

In [16]:
df2

,Model,Mean Absolute Error,Mean Squared Error,RMSE,R-squared
0,LinearRegression(),0.041822,0.004637,0.068098,0.546140
1,Ridge(alpha=0.5),0.041820,0.004637,0.068098,0.546141
2,Lasso(alpha=0.1),0.066582,0.009231,0.096081,0.096518
3,KNeighborsRegressor(n_neighbors=2),0.017831,0.001135,0.033693,0.888899
4,"(DecisionTreeRegressor(max_features=1.0, rando...",0.011636,0.000383,0.019582,0.962471
5,DecisionTreeRegressor(),0.014329,0.000670,0.025878,0.934457
6,SVR(),0.078155,0.009942,0.099712,0.026940
7,([DecisionTreeRegressor(criterion='friedman_ms...,0.022052,0.001055,0.032478,0.896764
8,"XGBRegressor(base_score=None, booster=None, ca...",0.013658,0.000446,0.021125,0.956323
9,ExplainableBoostingRegressor(),0.017983,0.000751,0.027404,0.926501


## Backward

In [17]:
import pandas as pd
x_train = pd.read_csv('X_train.csv')
x_test = pd.read_csv('X_val.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_val.csv')

In [18]:
x_train = x_train[models_backward]
x_train.head(3)

,country,team,season,md,outcome,win%,goals,gd,avg_starter_age
0,4,154,2018,13,1,0.076923,0.2,0.500000,0.531856
1,4,179,2010,19,1,0.105263,0.1,0.500000,0.443609
2,2,31,2006,6,0,0.500000,0.0,0.444444,0.511278


In [19]:
x_test = x_test[models_backward]
x_test.head(3)

,country,team,season,md,outcome,win%,goals,gd,avg_starter_age
0,3,91,2015,24,0,0.208333,0.0,0.388889,0.562753
1,0,176,2004,8,2,0.375000,0.1,0.555556,0.517544
2,2,103,2015,3,0,0.333333,0.1,0.388889,0.444079


Set up loop to train each of the models on the features identified in backward selection

In [20]:
model_eval = []
for model in models:
  model.fit(x_train, y_train.values.ravel())
  predictions = model.predict(x_test)
  type(predictions)
  # Calculate the absolute errors
  errors = abs(predictions - y_test.values.ravel())
  # Calculate model evaluation metrics
  mae = mean_absolute_error(y_test, predictions)
  mse = mean_squared_error(y_test, predictions)
  rmse = sqrt(mse)
  r2 = r2_score(y_test, predictions)
  model_eval.append({'Model': model, 'Mean Absolute Error': mae, 'Mean Squared Error': mse, 'RMSE': rmse,'R-squared': r2})

df3 = pd.DataFrame(model_eval)

In [21]:
df3

,Model,Mean Absolute Error,Mean Squared Error,RMSE,R-squared
0,LinearRegression(),0.054219,0.006102,0.078113,0.402836
1,Ridge(alpha=0.5),0.054218,0.006102,0.078113,0.402840
2,Lasso(alpha=0.1),0.066582,0.009231,0.096081,0.096518
3,KNeighborsRegressor(n_neighbors=2),0.021144,0.001597,0.039959,0.843733
4,"(DecisionTreeRegressor(max_features=1.0, rando...",0.015211,0.000946,0.030765,0.907369
5,DecisionTreeRegressor(),0.018653,0.001452,0.038099,0.857941
6,SVR(),0.079707,0.010237,0.101179,-0.001910
7,([DecisionTreeRegressor(criterion='friedman_ms...,0.032000,0.002554,0.050538,0.750030
8,"XGBRegressor(base_score=None, booster=None, ca...",0.017337,0.000811,0.028475,0.920647
9,ExplainableBoostingRegressor(),0.023824,0.001301,0.036064,0.872713
